In [ ]:
from mini_segment_anything import build_sam_yoso_r50
from segment_anything import sam_model_registry, SamPredictor

# sam-huge
sam_checkpoint = './sam_vit_h_4b8939.pth'
model_type = 'vit_h'
sam_huge = sam_model_registry[model_type](checkpoint=sam_checkpoint).to('cuda')
predictor_huge = SamPredictor(sam_huge)

# sam-mini
sam_checkpoint = './sam_yoso_r50_13a999.pth'
sam_mini = build_sam_yoso_r50(checkpoint=sam_checkpoint).to('cuda')
predictor_mini = SamPredictor(sam_mini)

In [ ]:
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

In [ ]:
image = cv2.imread('./images/truck.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_point = np.array([[600, 655]])
input_label = np.array([1])
input_box = np.array([400, 600, 700, 900])

In [ ]:
start = time.time()
predictor_huge.set_image(image)
end = time.time()

masks, scores, logits = predictor_huge.predict(
    point_coords=input_point,
    point_labels=input_label,
    # box=input_box[None, :],
    multimask_output=False,
)

plt.figure(figsize=(10,10))
plt.imshow(image)
show_mask(masks, plt.gca())
show_points(input_point, input_label, plt.gca())
# show_box(input_box, plt.gca())
plt.axis('on')
plt.title("SAM_huge time: " + str(end - start) + "s")
plt.show()

In [ ]:
start = time.time()
predictor_mini.set_image(image)
end = time.time()

masks, scores, logits = predictor_mini.predict(
    point_coords=input_point,
    point_labels=input_label,
    # box=input_box[None, :],
    multimask_output=False,
)

plt.figure(figsize=(10,10))
plt.imshow(image)
show_mask(masks, plt.gca())
show_points(input_point, input_label, plt.gca())
# show_box(input_box, plt.gca())
plt.title("SAM_mini time: " + str(end - start) + "s")
plt.axis('on')
plt.show() 